# Assignment 4 – Robot in a maze

In the  fourth assignment, you will delve into the application of RL algorithms to address the real-world challenge of navigating a robot through a maze dubbed 'robot in a maze'. The primary objectives of this assignment include:

 -   Formalizing a practical problem into a Markov Decision Process (MDP).
 -   Gaining familiarity with the OpenAI Gym framework (recently renamed as Gymnasium) and utilizing it to implement RL agents.
 -   Applying SARSA and Q-learning algorithms to solve the 'robot in a maze' MDP problem.
 -   Evaluating the outcomes of the reinforcement learning process and interpreting your findings.
 -   Reflecting on the distinctions between the two types of RL algorithms employed.

By accomplishing these objectives, you will not only enhance your understanding of RL algorithms but also develop practical skills in formulating and solving complex problems in the context of autonomous navigation within a maze.

In this assignment, you will be developing a robot to navigate its way through a maze. The project is divided into three parts (5 subtasks).

  1. In the first part, you will familiarize yourself with the OpenAI Gym/Gymnasium framework.
  2. In the second part, we have implemented the environment for you based on the Gym/Gymnasium framework. Your tasks include:  
     2.1. formalizing the problem as an MDP model,    
     2.2. implementing your own RL agents, and      
     2.3. training them to find the shortest route out of a maze.
  3. Finally, in the third part, you will evaluate and interpret the results obtained from the implemented RL agents


In [1]:
#!pip install gymnasium

We are now going to explain how the RL framework of gym works. 
- An **ENVIRONMENT**, 
- You also have an **AGENT**,
- In MDP problems, such as ours, the **ENVIRONMENT** provides an **OBSERVATION**, which represents the state of the **ENVIRONMENT** at the current moment.
- The agent takes an **ACTION** based on its **OBSERVATION**,
<!-- When a single **ACTION** is chosen and fed to our **ENVIRONMENT**, the **ENVIRONMENT** measures how good the action was taken and produces a **REWARD**, which is usually a numeric value. -->
- When the agent takes an ACTION, the ENVIRONMENT assesses the effectiveness of the action and generates a REWARD, which is usually a numeric value.

Please read the 'Basic usage' https://gymnasium.farama.org/content/basic_usage/ for better understanding the framework.  And do not forget import gymnasium before running other codes.

In [1]:
import gymnasium as gym

 ## 2. Tasks

Next, you will tackle a practical Markov Decision Process (MDP) problem, the 'robot in a maze,' based on the gym framework. Your task involves implementing an RL agent and training it to discover the shortest route to achieve the maze goal. In this MDP, the environment is represented as a grid world (a maze), with the agent being a robot. At each time step, the robot begins at a random location and can move within the grid world. The overarching objective is to find the way out, reaching the final location. Consequently, you will need to identify a fixed goal position within the maze.

### 2.1 Model the practical task into a MDP

To solve a RL problem, we start with formalizing the problem into a MDP model. Please describe this MDP model in your report. 

Notice: No empricial data provided in this assignment, so the point of 'data description and exploration' will be given to this step. 

While exploring your MDP model, you shall think about questions such as:
- What is the environment? How does it look like?
- What simulated data can your RL agent observe from the environment? How does it look like?
- Which data is considered as the state? Which data is considered as the reward?

#### Set up the environment

There is no need to implement your own environment; you should use the environment provided in the file **environment.py**. However, please ensure to take a look at it so that you understand the inner workings of this environment.

The core gym interface is **Env**, which serves as the unified environment interface. The following are the Env methods you should be familiar with:

- reset(self): Reset the environment's state and return the observation.
- step(self, action): Advance the environment by one timestep and return the observation, reward, done, and info.
- render(self, mode='rgb_array'): Render one frame of the environment. The default mode will produce something human-friendly, such as a pop up window. However, in this assignment, there is no need to create a pop-up window.

Please note that you need to install the [mazelab](https://github.com/yupei-du/mazelab.git) package, from **Yupei Du's** repository, to run the environment (a file with required packages is also provided). If you run the cell below for the first time, make sure to restart the IPython notebook at least once.


In [3]:
#!git clone https://github.com/yupei-du/mazelab.git
#!pip install -e mazelab
#!pip install pandas
#!pip install seaborn

You can now check whether the required packages (e.g. mazela, pandas, tqdm, seaborn) are installed. Please install the ones are missing. 

ATTENTION: To run the given code, please use the python version 3.7-3.9, and the numpy version < 1.23

In [4]:
# # Check Python version
# import sys
# print(f"Python version: {sys.version}")

# # Check NumPy version
# import numpy as np
# print(f"NumPy version: {np.__version__}")

#conda list

We also provide a few helper functions to make it easier to debug your agents. 
 - `animate_run` will enable you to see the agent's behavior. It takes a list of images which can be produced by the `env.render` function of the environment
 - `visualize_agent_brain` will provide you with a way to visualize the agents learned q_table. Use it after you have implemented and trained your agents. The first plot will show the highest q-value per state (position on the map) and the second will tell you which action the agent would choose at that state/position. It takes the environment and the agent as input.

Below you will find a basic example of how the animation function works. Please notice that: whenever you **reset()** the environment, the agent will start at a random position (a different state). 

In [ ]:
# The helper functions

from IPython import get_ipython
import random
from mazelab.generators import random_maze, morris_water_maze
import matplotlib.pyplot as plt
from IPython import display
from mazelab.solvers import dijkstra_solver
import seaborn as sns
from tqdm.notebook import tqdm
from environment import TaskEnv
from typing import Tuple, List
import itertools as it
import numpy as np
import pandas as pd

def animate_run(data:List[np.ndarray]):
    init_img = data[0]
    remaining_img = data[1:]
    img_container = plt.imshow(init_img)  # only call this once
    for img in remaining_img:
        img_container.set_data(img)  # just update the data
        display.display(plt.gcf())
        display.clear_output(wait=True)


def visualize_agent_brain(agent, env: TaskEnv):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    ax1.set_title("Highest state value at position (x,y)")
    state_value_map = agent.q_table.max(axis=2)
    sns.heatmap(state_value_map, ax=ax1)

    ax2.set_title("Chosen action at position (x,y)")
    n = env.action_space.n + 1
    path = env.maze.objects.free.positions
    decisions_map = np.array([[x_, y_, agent.select_action([x_, y_]) + 1] for x_, y_ in path])
    state_action_map = np.zeros_like(agent.q_table.max(axis=2))
    state_action_map[decisions_map[:, 0], decisions_map[:, 1]] = decisions_map[:, 2]
    cmap = sns.color_palette("viridis", n)
    sns.heatmap(state_action_map, cmap=cmap, ax=ax2)
    colorbar = ax2.collections[0].colorbar
    r = (colorbar.vmax) - colorbar.vmin
    colorbar.set_ticks([colorbar.vmin + r / n * (0.5 + i) for i in range(n)])
    colorbar.set_ticklabels(['N/A', 'north', 'south', 'west', 'east'])
    fig.tight_layout()
    return plt.show()


env = TaskEnv()
env.reset()
impassable_array = env.unwrapped.maze.to_impassable()
motions = env.unwrapped.motions
start = env.unwrapped.maze.objects.agent.positions[0]
goal = env.unwrapped.maze.objects.goal.positions[0]
actions = dijkstra_solver(impassable_array, motions, start, goal)
print(actions)

imgs = []
rewards = 0.0
for action in actions:
    _, reward, _, _ = env.step(action)
    rewards += reward
    imgs.append(env.render("rgb_array"))
print(rewards)

animate_run(imgs)
plt.show()

### 2.2  Implement the agents 

In this part, you are expected to implement two RL agents. 

- Agent 1 uses the Q-learning algorithm to learn the optimal solution
- Agent 2 uses the SARSA algorithm to learn the optimal solution. To decide the action to take at each time step,  this agent uses the epsilon greedy action selection.

Here, we have also provided an example agent: the Random Agent.  It follows a random policy to move at each step (randomly selecting an action). You can use this example agent as a baseline to evaluate your agents.


In [25]:
# Random agent
class RandomAgent():
    def __init__(self,
                 env: TaskEnv,
                 exploration_rate: float = None,
                 learning_rate: float = None,
                 discount_factor: float = None) -> int:
        self.env = env
        self.epsilon = 1  # A random agent "explores" always, so epsilon will be 1
        self.alpha = 0  # A random agent never learns, so there's no need for a learning rate
        self.gamma = 0  # A random agent does not update it's q-table. Hence, it's zero.
        self.q_table = np.zeros(env.observation_space.shape + (env.action_space.n, ), dtype=float)
        self.actions = env.action_space

    def select_action(self, state: Tuple[int, int], use_greedy_strategy: bool = False) -> int:
        if not use_greedy_strategy:
            if random.random() < self.epsilon:
                next_action = self.actions.sample()
                return next_action

        x, y = state
        max_val = np.max(self.q_table[x, y, :])
        find_max_val = np.where(self.q_table[x, y, :] == max_val)
        next_action = np.random.choice(find_max_val[0])
        return next_action

    def learn(self, state, action, next_state, reward, done):
        return None
    
    def train(self, n_episodes=10_000):
        rewards_per_episode = []
        for episode in tqdm(range(n_episodes)):
            obs = self.env.reset()
            done = False
            total_rewards = 0

            while not done:
                action = self.select_action(obs)
                next_obs, reward, done, info = self.env.step(action)
                total_rewards += reward
                self.learn(obs, action, next_obs, reward, done)

                obs = next_obs

            rewards_per_episode.append(total_rewards)
        
        return rewards_per_episode

    def reset(self):
        self.q_table = np.zeros(env.observation_space.shape + (env.action_space.n, ), dtype=float)


# TODO: implement two agents   

In [8]:
# Q-learning agent
class QLearningAgent():
    def __init__(self,
                 env: TaskEnv,
                 exploration_rate: float = None,
                 learning_rate: float = None,
                 discount_factor: float = None) -> int:
        self.env = env
        self.epsilon = exploration_rate  # A random agent "explores" always, so epsilon will be 1
        self.alpha = learning_rate  # A random agent never learns, so there's no need for a learning rate
        self.gamma = discount_factor  # A random agent does not update it's q-table. Hence, it's zero.
        self.q_table = np.zeros(env.observation_space.shape + (env.action_space.n, ), dtype=float)
        self.actions = env.action_space

    def select_action(self, state: Tuple[int, int], use_greedy_strategy: bool = False) -> int:
        if not use_greedy_strategy:
            if random.random() < self.epsilon:
                next_action = self.actions.sample()
                return next_action

        x, y = state
        max_val = np.max(self.q_table[x, y, :])
        find_max_val = np.where(self.q_table[x, y, :] == max_val)
        next_action = np.random.choice(find_max_val[0])
        return next_action

    def learn(self, state, action, next_state, reward, done):
        x, y = state
        next_x, next_y = next_state
        old_value = self.q_table[x, y, action]
        next_max = np.max(self.q_table[next_x, next_y, :])
        
        self.q_table[x, y, action] = old_value + self.alpha * (reward + self.gamma * next_max - old_value)
    
    def train(self, n_episodes=10_000):
        rewards_per_episode = []
        for episode in tqdm(range(n_episodes)):
            obs = self.env.reset()
            done = False
            total_rewards = 0

            while not done:
                action = self.select_action(obs)
                next_obs, reward, done, info = self.env.step(action)
                total_rewards += reward
                self.learn(obs, action, next_obs, reward, done)

                obs = next_obs

            rewards_per_episode.append(total_rewards)
        
        return rewards_per_episode

    def reset(self):
        self.q_table = np.zeros(env.observation_space.shape + (env.action_space.n, ), dtype=float)

In [7]:
# Sarsa agent
class SarsaAgent():
    def __init__(self,
                 env: TaskEnv,
                 exploration_rate: float = None,
                 learning_rate: float = None,
                 discount_factor: float = None) -> int:
        self.env = env
        self.epsilon = exploration_rate  # A random agent "explores" always, so epsilon will be 1
        self.alpha = learning_rate  # A random agent never learns, so there's no need for a learning rate
        self.gamma = discount_factor  # A random agent does not update it's q-table. Hence, it's zero.
        self.q_table = np.zeros(env.observation_space.shape + (env.action_space.n, ), dtype=float)
        self.actions = env.action_space

    def select_action(self, state: Tuple[int, int], use_greedy_strategy: bool = False) -> int:
        if not use_greedy_strategy:
            if random.random() < self.epsilon:
                next_action = self.actions.sample()
                return next_action

        x, y = state
        max_val = np.max(self.q_table[x, y, :])
        find_max_val = np.where(self.q_table[x, y, :] == max_val)
        next_action = np.random.choice(find_max_val[0])
        return next_action
 

    def learn(self, state, action, next_state, reward, next_action, done):
        x, y = state
        next_x, next_y = next_state
        
        old_value = self.q_table[x, y, action]
        next_value = self.q_table[next_x, next_y, next_action] if not done else 0
        
        self.q_table[x, y, action] = old_value + self.alpha * (reward + self.gamma * next_value - old_value)
    
    def train(self, n_episodes=10_000):
        rewards_per_episode = []
        for episode in tqdm(range(n_episodes)):
            obs = self.env.reset()
            done = False
            total_rewards = 0

            action = self.select_action(obs)
            while not done:
                next_obs, reward, done, info = self.env.step(action)
                total_rewards += reward
                next_action = self.select_action(next_obs) 
                
                self.learn(obs, action, next_obs, reward, next_action, done)

                obs = next_obs
                action = next_action

            rewards_per_episode.append(total_rewards)
        return rewards_per_episode
                
    def reset(self):
        self.q_table = np.zeros(env.observation_space.shape + (env.action_space.n, ), dtype=float)

In [ ]:
################# TRAINING Agents #################
# Handige link: https://gymnasium.farama.org/introduction/train_agent/

def_exploration = 0.2
def_learning = 0.01
def_discount = 0.95
train_env = TaskEnv()
q_agent = QLearningAgent(train_env, 
                       exploration_rate = def_exploration, 
                       learning_rate    = def_learning, 
                       discount_factor  = def_discount
                      )

sarsa_agent = SarsaAgent(train_env, 
                       exploration_rate = def_exploration, 
                       learning_rate    = def_learning, 
                       discount_factor  = def_discount
                      )

q_agent.train()
sarsa_agent.train()


### 2.3 Run the simulation

Now, you write code for running a simulation. In each run, you shall setup the epsilon parameter. 

In [10]:
# TODO: run the simulation
def runSim(agent):
    obs = agent.env.reset()
    done = False
    total_reward = 0.0
    imgs = []

    while not done:
        # Remove exploration during testing
        action = agent.select_action(obs, use_greedy_strategy=True)
        
        next_obs, reward, done, info = agent.env.step(action)
        print(reward)
        total_reward += reward

        imgs.append(agent.env.render("rgb_array"))

        obs = next_obs

    animate_run(imgs)
    plt.show()
    
    visualize_agent_brain(agent, agent.env)

    print("Total reward:", total_reward)

    return total_reward


In [ ]:
runSim(q_agent)

In [ ]:
runSim(sarsa_agent)

 ## 3. Play with parameters and analyse results
 
Finally, you will describe, evaluate, and interpret the results obtained from the two RL agents. Additionally, compare your agents with the provided Random Agent. Feel free to utilize the provided helper functions for evaluating your agents. Some important points are:

- Both quantified evaluation and human evaluation are needed in the report. The quantified evaluation should focus on the measurement of reward. In the human evaluation, you can use the provided visual tools to interpret your results. Your report should include at least one plot presenting comparable measures for the different agents.

- While evaluating the results of Agent 2 (with SARSA algorithm), please try at least 2 different values of **epsilon** (expect 0) and discuss the influence of different epsilon values on the results. In the end, please identify a reasonable epsilon value that could balance the exploration and exploitation, then fix this value for comparing the two agents. Present your trails and results in the report.

- In the report, you also need to parcitularly describe and discuss the similarity and difference of results from two RL agents (hint: on-policy VS off-policy). For this, please make sure that the compared results are obtained from the same environment (the same maze for two different agents). Also, while evaluating the results of two agents, please try at least 2 different values of **gamma**. In this way, you could discuss the influence of this discount factor in your report. 

- Please run the simulation multiple times and average the results for all your findings.


In [21]:
####################### FUNCTIONS FOR PARAMETER TUNING ################################################

# Trains X agents in the same maze and visualizes the average state values
def visualize_average_state_values(agent, X=5, algorithm="Empty"):
    total_state_values = np.zeros_like(agent.q_table.max(axis=2))
    
    for _ in range(X):
        agent.reset() # Reset q-table
        agent.train()  # Train the agent again
        total_state_values += agent.q_table.max(axis=2)  # Add state values

    avg_state_values = total_state_values / X

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_title(f"Average Highest State Value at position (x,y)")
    sns.heatmap(avg_state_values, ax=ax)

    plt.show()


########################################################################################################


def train_multiple(agent, exploration_rate=0.1, discount_rate=0.95, n_episodes=10_000, N=5):
    agent.epsilon = exploration_rate
    agent.gamma = discount_rate
    for i in range(N):
        agent.reset()
        total_rewards_per_episode = agent.train(n_episodes = n_episodes)
    avg_total_rewards_per_episode = [elem / N for elem in total_rewards_per_episode]
    return avg_total_rewards_per_episode


#########################################################################################################

def plot_exploration_test_comparison(epsilons, rewards_list, algorithm="Empty"):
    plt.figure(figsize=(10, 6))
    
    for i, rewards in enumerate(rewards_list):
        smoothed_rewards = np.convolve(rewards, np.ones(100), mode='valid')
        plt.plot(smoothed_rewards, label=f"Epsilon = {epsilons[i]}")
    
    plt.xlabel("Episode")
    plt.ylabel("Mean Total Reward")
    plt.title(f"Effect of Exploration Rate on Total Reward with {algorithm}")
    plt.legend()
    plt.show()

def plot_discount_test_comparison(discounts, rewards_list, algorithm="Empty"):
    plt.figure(figsize=(10, 6))
    
    for i, rewards in enumerate(rewards_list):
        smoothed_rewards = np.convolve(rewards, np.ones(100), mode='valid')
        plt.plot(smoothed_rewards, label=f"Discount = {discounts[i]}")
    
    plt.xlabel("Episode")
    plt.ylabel("Mean Total Reward")
    plt.title(f"Effect of Discount Rate on Total Reward with {algorithm}")
    plt.legend()
    plt.show()

def optimize(agent):
    agent.epsilon = 0.1
    agent.gamma = 0.95
    return agent


In [ ]:
# Effect of exploration rate on SARSA and Q-learning
epsilons = [0.1, 0.3, 0.5]
exploration_episodes = 15_000
rewards_sarsa_0_1 = train_multiple(sarsa_agent, exploration_rate=epsilons[0], n_episodes=exploration_episodes)
rewards_sarsa_0_3 = train_multiple(sarsa_agent, exploration_rate=epsilons[1], n_episodes=exploration_episodes)
rewards_sarsa_0_5 = train_multiple(sarsa_agent, exploration_rate=epsilons[2], n_episodes=exploration_episodes)

rewards_qlearning_0_1 = train_multiple(q_agent, exploration_rate=epsilons[0], n_episodes=exploration_episodes)
rewards_qlearning_0_3 = train_multiple(q_agent, exploration_rate=epsilons[1], n_episodes=exploration_episodes)
rewards_qlearning_0_5 = train_multiple(q_agent, exploration_rate=epsilons[2], n_episodes=exploration_episodes)

plot_exploration_test_comparison(epsilons, [rewards_sarsa_0_1, rewards_sarsa_0_3, rewards_sarsa_0_5], algorithm="SARSA")
plot_exploration_test_comparison(epsilons, [rewards_qlearning_0_1, rewards_qlearning_0_3, rewards_qlearning_0_5], algorithm="Q-Learning")


In [ ]:
# Effect of discount rate on SARSA and q-learning
discounts = [0.90, 0.95, 0.99]
discount_episodes = 10_000
rewards_sarsa_0_90 = train_multiple(sarsa_agent, discount_rate=discounts[0], n_episodes=discount_episodes)
rewards_sarsa_0_95 = train_multiple(sarsa_agent, discount_rate=discounts[1], n_episodes=discount_episodes)
rewards_sarsa_0_99 = train_multiple(sarsa_agent, discount_rate=discounts[2], n_episodes=discount_episodes)

rewards_qlearning_0_90 = train_multiple(q_agent, discount_rate=discounts[0], n_episodes=discount_episodes)
rewards_qlearning_0_95 = train_multiple(q_agent, discount_rate=discounts[1], n_episodes=discount_episodes)
rewards_qlearning_0_99 = train_multiple(q_agent, discount_rate=discounts[2], n_episodes=discount_episodes)

plot_discount_test_comparison(discounts, [rewards_sarsa_0_90, rewards_sarsa_0_95, rewards_sarsa_0_99], algorithm="SARSA")
plot_discount_test_comparison(discounts, [rewards_qlearning_0_90, rewards_qlearning_0_95, rewards_qlearning_0_99], algorithm="Q Learning")


In [ ]:
# Show q-table for optimal hyperparameter
q_agent_optimal = optimize(q_agent)
visualize_average_state_values(q_agent_optimal)

In [ ]:
# Show q-table for optimal hyperparameters
sarsa_agent_optimal = optimize(sarsa_agent)
visualize_average_state_values(sarsa_agent_optimal)

In [ ]:
# Random Agent
random_agent = RandomAgent(train_env)
rewards_randomagent_0_1 = train_multiple(random_agent, exploration_rate=epsilons[0])
rewards_randomagent_0_3 = train_multiple(random_agent, exploration_rate=epsilons[1])
rewards_randomagent_0_5 = train_multiple(random_agent, exploration_rate=epsilons[2])
rewards_randomagent_0_90 = train_multiple(random_agent, discount_rate=discounts[0])
rewards_randomagent_0_95 = train_multiple(random_agent, discount_rate=discounts[1])
rewards_randomagent_0_99 = train_multiple(random_agent, discount_rate=discounts[2])


plot_exploration_test_comparison(epsilons, [rewards_randomagent_0_1, rewards_randomagent_0_3, rewards_randomagent_0_5], algorithm="Random")
plot_discount_test_comparison(discounts, [rewards_randomagent_0_90, rewards_randomagent_0_95, rewards_randomagent_0_99], algorithm="Random")


# Bonus Tasks 

We would like to challenge you with the following bonus task. For each task that is successfully completed, you may obtain max. 1 extra point. 

1. Implement a third RL agent using another RL algorithm (e.g. Monte Carlo methods, Expected SARSA or even neural network-based ones) and discuss your findings. Compare this third agent with the above ones and explain why this is a better (or worse) RL algorithm. You are allowed to reuse exsiting packages, but please cite them, test them in advance, and make sure that you can explain the used algorithm using your own words.

2. Can you explore and show other evaluation results? If so, implement and present one extra result (e.g. a plot). And please explain why it is a good evaluation method for our task or how it shows the difference between two RL agents/algorithms.